In [2]:
import pyspark
from pyspark.sql import SparkSession
import os


conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
    
    # first we will define the packages that we need. Iceberg Spark runtime
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.2.0,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
        
    # This property allows us to add any extensions that we want to use
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
    
    # configures a new catalog to a particular implementation of SparkCatalog
        .set('spark.sql.catalog.glue', 'org.apache.iceberg.spark.SparkCatalog')
    
    # particular type of catalog we are using
        .set('spark.sql.catalog.glue.catalog-impl', 'org.apache.iceberg.aws.glue.GlueCatalog')
    
    # engine writes to the warehouse
        .set('spark.sql.catalog.glue.warehouse', 's3://bucket/warehouse/')
    
    # changes IO impl of catalog, mainly for changing writing data to object storage
        .set('spark.sql.catalog.glue.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
)

## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

:: loading settings :: url = jar:file:/home/docker/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/docker/.ivy2/cache
The jars for the packages stored in: /home/docker/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-02107c15-1fb0-487a-a590-a2f0a2a9dc3f;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.2.0 in central
	found software.amazon.awssdk#bundle;2.17.178 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.17.178 in central
	found software.amazon.awssdk#utils;2.17.178 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found software.amazon.awssdk#annotations;2.17.178 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found software.amazon.awssdk#http-client-spi;2.17.178 in central
	found software

23/05/04 21:40:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/04 21:40:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Spark Running


# Method 1: WAP ID

In [77]:
spark.sql("SELECT * FROM glue.test.churn").toPandas()

,State,Account_length,Area_code,International_plan,Voicemail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,FALSE
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.4,12.7,6,3.43,4,TRUE
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,TRUE
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,FALSE
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.0,1,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,SC,79,415,No,No,0,134.7,98,22.9,189.7,68,16.12,221.4,128,9.96,11.8,5,3.19,2,FALSE
931,AZ,192,415,No,Yes,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2,FALSE
932,WV,68,415,No,No,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3,FALSE
933,RI,28,510,No,No,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2,FALSE


# WRITE Stage:

In [47]:
import uuid
session_id_new = uuid.uuid4().hex
print(session_id_new)

4462ae45b5664c2cb8a95032ec910892


In [48]:
spark.sql(
    """ALTER TABLE glue.test.churn
       SET TBLPROPERTIES ('write.wap.enabled'='true')"""
)

DataFrame[]

In [49]:
spark.conf.set('spark.wap.id', session_id_new)

In [50]:
spark.sql(
    """CREATE OR REPLACE TEMPORARY VIEW chview USING csv
            OPTIONS (path "churn_etl.csv", header true)"""
)
spark.sql("INSERT INTO glue.test.churn SELECT * FROM chview")

DataFrame[]

In [52]:
spark.sql("SELECT COUNT(*) FROM glue.test.churn").toPandas()

,count(1)
0,935


# AUDIT stage:

In [53]:
# New:
spark.sql("SELECT snapshot_id FROM glue.test.churn.snapshots WHERE summary['wap.id'] = '4462ae45b5664c2cb8a95032ec910892'").toPandas()

23/05/03 18:44:04 WARN SparkScanBuilder: Failed to check if IsNotNull(summary) can be pushed down: Cannot find field 'summary' in struct: struct<>


,snapshot_id
0,7869769243560997710


In [54]:
spark.sql("SELECT * FROM glue.test.churn VERSION AS OF 7869769243560997710").toPandas()

,State,Account_length,Area_code,International_plan,Voicemail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,FALSE
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.4,12.7,6,3.43,4,TRUE
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,TRUE
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,FALSE
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.0,1,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,WI,114,415,No,Yes,26,137.1,88,23.31,155.7,125,13.23,247.6,94,11.14,11.5,7,3.11,2,FALSE
936,AL,106,408,No,Yes,29,83.6,131,14.21,203.9,131,17.33,229.5,73,10.33,8.1,3,2.19,1,FALSE
937,VT,60,415,No,No,0,193.9,118,32.96,85.0,110,7.23,210.1,134,9.45,13.2,8,3.56,3,FALSE
938,WV,159,415,No,No,0,169.8,114,28.87,197.7,105,16.8,193.7,82,8.72,11.6,4,3.13,1,FALSE


In [76]:
from pyspark.sql.functions import isnan, when, count, col

df_qc = spark.read.option("snapshot-id", 7869769243560997710) \
        .format("iceberg") \
        .load("glue.test.churn") \

# Check for NULL values in all columns
null_counts = df_qc.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_qc.columns])

if any([row[c] > 0 for row in null_counts.collect() for c in row]):
    print("There are NULL values in the DataFrame.")
else:
    print("There are no NULL values in the DataFrame.")

There are no NULL values in the DataFrame.


# PUBLISH stage:

In [78]:
spark.sql("CALL glue.system.cherrypick_snapshot('test.churn', 7869769243560997710)").show()

23/05/03 19:24:25 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up
+-------------------+-------------------+
| source_snapshot_id|current_snapshot_id|
+-------------------+-------------------+
|7869769243560997710|7869769243560997710|
+-------------------+-------------------+



In [79]:
# Validate:
spark.sql("SELECT * FROM glue.test.churn").toPandas()

,State,Account_length,Area_code,International_plan,Voicemail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,WI,114,415,No,Yes,26,137.1,88,23.31,155.7,125,13.23,247.6,94,11.14,11.5,7,3.11,2,FALSE
1,AL,106,408,No,Yes,29,83.6,131,14.21,203.9,131,17.33,229.5,73,10.33,8.1,3,2.19,1,FALSE
2,VT,60,415,No,No,0,193.9,118,32.96,85.0,110,7.23,210.1,134,9.45,13.2,8,3.56,3,FALSE
3,WV,159,415,No,No,0,169.8,114,28.87,197.7,105,16.8,193.7,82,8.72,11.6,4,3.13,1,FALSE
4,CT,184,510,Yes,No,0,213.8,105,36.35,159.6,84,13.57,139.2,137,6.26,5.0,10,1.35,2,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,SC,79,415,No,No,0,134.7,98,22.9,189.7,68,16.12,221.4,128,9.96,11.8,5,3.19,2,FALSE
936,AZ,192,415,No,Yes,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2,FALSE
937,WV,68,415,No,No,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3,FALSE
938,RI,28,510,No,No,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2,FALSE


# Method 2: WAP BRANCH

In [3]:
spark.sql("SELECT * FROM glue.test.salesnew").toPandas()

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


,id,name,product,price,date
0,1,Selinda Rheaume,Wine - Prosecco Valdobiaddene,10.31,10/20/2022
1,2,Wynnie Gozzard,"Wine - Red, Wolf Blass, Yellow",20.05,10/20/2022
2,3,Patten Whitter,Crackers - Soda / Saltins,39.75,10/20/2022
3,4,Hulda Eslie,Roe - White Fish,37.10,10/20/2022
4,5,Chrystal Haggie,Wine - Delicato Merlot,35.82,10/20/2022
...,...,...,...,...,...
998,996,Anna-diana Krystof,"Pepper - White, Whole",59.68,10/21/2022
999,997,Mariel Candwell,Corn Meal,32.84,10/21/2022
1000,998,Jervis Isabell,Cheese - Cheddar With Claret,None,10/21/2022
1001,999,Elsi Tookill,Appetizer - Asian Shrimp Roll,94.96,10/21/2022


In [4]:
spark.sql("ALTER TABLE glue.test.salesnew CREATE BRANCH ETL_0405").show()

23/05/04 22:42:16 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up
++
||
++
++



In [5]:
spark.sql("SELECT * FROM glue.test.salesnew VERSION AS OF 'ETL_0405'").toPandas()

,id,name,product,price,date
0,498,Elston Bage,Halibut - Fletches,20.61,10/21/2022
1,499,Daryl Maffioni,Cornstarch,44.13,10/21/2022
2,500,Glad Berre,Sour Puss Sour Apple,45.91,10/21/2022
3,501,Marjy Riha,"Beef - Ox Tail, Frozen",20.42,10/21/2022
4,502,Augie Hancill,Tomatoes - Hot House,19.02,10/21/2022
...,...,...,...,...,...
998,496,Winny McGlone,Pasta - Fusili Tri - Coloured,67.78,10/20/2022
999,497,Hadleigh Ellinor,Oats Large Flake,58.19,10/20/2022
1000,498,Kimberlee Hancill,"Soup - Knorr, Country Bean",81.64,10/20/2022
1001,499,Anet Scaife,Soup - French Onion,98.51,10/20/2022


# WRITE stage:

In [8]:
spark.sql(
    """ALTER TABLE glue.test.salesnew
       SET TBLPROPERTIES ('write.wap.enabled'='true')"""
)

DataFrame[]

In [9]:
spark.conf.set('spark.wap.branch', 'ETL_0405')

In [10]:
spark.sql(
    """CREATE OR REPLACE TEMPORARY VIEW salesetlnew USING csv
            OPTIONS (path "sales_ETL_new.csv", header true)"""
)
spark.sql("INSERT INTO glue.test.salesnew SELECT * FROM salesetlnew")

DataFrame[]

In [11]:
spark.sql("SELECT * FROM glue.test.salesnew").toPandas()

,id,name,product,price,date
0,498,Elston Bage,Halibut - Fletches,20.61,10/21/2022
1,499,Daryl Maffioni,Cornstarch,44.13,10/21/2022
2,500,Glad Berre,Sour Puss Sour Apple,45.91,10/21/2022
3,501,Marjy Riha,"Beef - Ox Tail, Frozen",20.42,10/21/2022
4,502,Augie Hancill,Tomatoes - Hot House,19.02,10/21/2022
...,...,...,...,...,...
1002,500,Bary Reap,Soup - Tomato Mush. Florentine,92.64,10/20/2022
1003,527,Haleigh Kirimaa,Samosa - Veg,75.04,10/21/2022
1004,528,Norbert Yegorov,Silicone Paper 16.5x24,None,10/21/2022
1005,529,Sibylla Guiness,Milk - Buttermilk,24.85,10/21/2022


In [12]:
spark.sql("SELECT * FROM glue.test.salesnew VERSION AS OF 'ETL_0405'").toPandas()

,id,name,product,price,date
0,527,Haleigh Kirimaa,Samosa - Veg,75.04,10/21/2022
1,528,Norbert Yegorov,Silicone Paper 16.5x24,None,10/21/2022
2,529,Sibylla Guiness,Milk - Buttermilk,24.85,10/21/2022
3,530,Chiquia Crippill,Table Cloth - 53x69 Colour,None,10/21/2022
4,498,Elston Bage,Halibut - Fletches,20.61,10/21/2022
...,...,...,...,...,...
1002,496,Winny McGlone,Pasta - Fusili Tri - Coloured,67.78,10/20/2022
1003,497,Hadleigh Ellinor,Oats Large Flake,58.19,10/20/2022
1004,498,Kimberlee Hancill,"Soup - Knorr, Country Bean",81.64,10/20/2022
1005,499,Anet Scaife,Soup - French Onion,98.51,10/20/2022


In [13]:
spark.sql("SELECT * FROM glue.test.salesnew.branch_ETL_0405").toPandas()

,id,name,product,price,date
0,498,Elston Bage,Halibut - Fletches,20.61,10/21/2022
1,499,Daryl Maffioni,Cornstarch,44.13,10/21/2022
2,500,Glad Berre,Sour Puss Sour Apple,45.91,10/21/2022
3,501,Marjy Riha,"Beef - Ox Tail, Frozen",20.42,10/21/2022
4,502,Augie Hancill,Tomatoes - Hot House,19.02,10/21/2022
...,...,...,...,...,...
1002,500,Bary Reap,Soup - Tomato Mush. Florentine,92.64,10/20/2022
1003,527,Haleigh Kirimaa,Samosa - Veg,75.04,10/21/2022
1004,528,Norbert Yegorov,Silicone Paper 16.5x24,None,10/21/2022
1005,529,Sibylla Guiness,Milk - Buttermilk,24.85,10/21/2022


In [14]:
spark.sql("SELECT * FROM glue.test.salesnew VERSION AS OF 'main'").toPandas()

,id,name,product,price,date
0,1,Selinda Rheaume,Wine - Prosecco Valdobiaddene,10.31,10/20/2022
1,2,Wynnie Gozzard,"Wine - Red, Wolf Blass, Yellow",20.05,10/20/2022
2,3,Patten Whitter,Crackers - Soda / Saltins,39.75,10/20/2022
3,4,Hulda Eslie,Roe - White Fish,37.10,10/20/2022
4,5,Chrystal Haggie,Wine - Delicato Merlot,35.82,10/20/2022
...,...,...,...,...,...
998,996,Anna-diana Krystof,"Pepper - White, Whole",59.68,10/21/2022
999,997,Mariel Candwell,Corn Meal,32.84,10/21/2022
1000,998,Jervis Isabell,Cheese - Cheddar With Claret,None,10/21/2022
1001,999,Elsi Tookill,Appetizer - Asian Shrimp Roll,94.96,10/21/2022


# Audit Stage:

In [15]:
from pyspark.sql.functions import isnan, when, count, col

# Load your DataFrame
df_qc_sales = spark.read.option("branch", "ETL_0405") \
        .format("iceberg") \
        .load("glue.test.salesnew") \

# Check for NULL values in all columns
null_counts_new = df_qc_sales.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_qc_sales.columns]).collect()[0]

# Check if any columns have NULL values
if any([null_counts_new[i] > 0 for i in range(len(null_counts_new))]):
    print("There are NULL values in the DataFrame.")
else:
    print("There are no NULL values in the DataFrame.")


There are NULL values in the DataFrame.


In [16]:
print(null_counts_new)

Row(id=0, name=0, product=0, price=56, date=0)


In [24]:
dp = df_qc_sales.toPandas()

In [27]:
mask = dp.isnull().any(axis=1)
null_rows = dp[mask]
null_rows

,id,name,product,price,date
18,516,Adelheid Darby,"Pork - Bacon, Double Smoked",None,10/21/2022
30,528,Norbert Yegorov,Silicone Paper 16.5x24,None,10/21/2022
32,530,Chiquia Crippill,Table Cloth - 53x69 Colour,None,10/21/2022
47,545,Farlee Parzis,Barramundi,None,10/21/2022
62,560,Saw Juorio,Instant Coffee,None,10/21/2022
64,562,Phyllis Foye,Bok Choy - Baby,None,10/21/2022
66,564,Kleon Carrabot,"Pasta - Rotini, Colour, Dry",None,10/21/2022
81,579,Ricca Jakubczyk,"Chicken - Leg, Fresh",None,10/21/2022
83,581,Alister Pardon,Coconut Milk - Unsweetened,None,10/21/2022
84,582,Innis Wynrehame,Pop Shoppe Cream Soda,None,10/21/2022


In [17]:
from pyspark.sql.functions import col, coalesce, percentile_approx, lit


# Calculate the median value for each column
medians = df_qc_sales.agg(*[percentile_approx(col(c), 0.5).alias(c) for c in df_qc_sales.columns])

# Replace null values with the calculated median
df_imputed = df_qc_sales.select([coalesce(col(c), lit(str(medians.first()[c]))).alias(c) for c in df_qc_sales.columns])

# Show the imputed DataFrame
df_imputed.show()


+---+-------------------+--------------------+-----+----------+
| id|               name|             product|price|      date|
+---+-------------------+--------------------+-----+----------+
|498|        Elston Bage|  Halibut - Fletches|20.61|10/21/2022|
|499|     Daryl Maffioni|          Cornstarch|44.13|10/21/2022|
|500|         Glad Berre|Sour Puss Sour Apple|45.91|10/21/2022|
|501|         Marjy Riha|Beef - Ox Tail, F...|20.42|10/21/2022|
|502|      Augie Hancill|Tomatoes - Hot House|19.02|10/21/2022|
|503|Clerkclaude Zottoli|Bag - Regular Kra...|40.28|10/21/2022|
|504|    Whitney Bonhill|    Cheese - Ricotta|93.65|10/21/2022|
|505| Carrissa Murrhaupt|   Tilapia - Fillets|44.14|10/21/2022|
|506|     Ira Saunderson|Appetizer - Assor...|37.30|10/21/2022|
|507|  Lisabeth Grealish|Shortbread - Cook...|24.39|10/21/2022|
|508|       Meggy Hanner|Ice Cream - Turtl...|46.10|10/21/2022|
|509|  Deloris Delacroix|      Sauce - Hoisin|33.65|10/21/2022|
|510|     Pate Corington|Meldea Green Te

In [18]:
df_imputed.toPandas()

,id,name,product,price,date
0,1,Selinda Rheaume,Wine - Prosecco Valdobiaddene,10.31,10/20/2022
1,2,Wynnie Gozzard,"Wine - Red, Wolf Blass, Yellow",20.05,10/20/2022
2,3,Patten Whitter,Crackers - Soda / Saltins,39.75,10/20/2022
3,4,Hulda Eslie,Roe - White Fish,37.10,10/20/2022
4,5,Chrystal Haggie,Wine - Delicato Merlot,35.82,10/20/2022
...,...,...,...,...,...
1002,1000,Morgan Lagden,"Lentils - Green, Dry",55.34,10/21/2022
1003,527,Haleigh Kirimaa,Samosa - Veg,75.04,10/21/2022
1004,528,Norbert Yegorov,Silicone Paper 16.5x24,55.34,10/21/2022
1005,529,Sibylla Guiness,Milk - Buttermilk,24.85,10/21/2022


In [19]:
null_counts_impute = df_imputed.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_imputed.columns]).collect()[0]

# Check if any columns have NULL values
if any([null_counts_impute[i] > 0 for i in range(len(null_counts_impute))]):
    print("There are NULL values in the DataFrame.")
else:
    print("There are no NULL values in the DataFrame.")

There are no NULL values in the DataFrame.


In [20]:
df_imputed.write.format("iceberg").mode("overwrite").save("glue.test.salesnew")

In [22]:
spark.sql("SELECT * FROM glue.test.salesnew").toPandas()

,id,name,product,price,date
0,498,Elston Bage,Halibut - Fletches,20.61,10/21/2022
1,499,Daryl Maffioni,Cornstarch,44.13,10/21/2022
2,500,Glad Berre,Sour Puss Sour Apple,45.91,10/21/2022
3,501,Marjy Riha,"Beef - Ox Tail, Frozen",20.42,10/21/2022
4,502,Augie Hancill,Tomatoes - Hot House,19.02,10/21/2022
...,...,...,...,...,...
1002,496,Winny McGlone,Pasta - Fusili Tri - Coloured,67.78,10/20/2022
1003,497,Hadleigh Ellinor,Oats Large Flake,58.19,10/20/2022
1004,498,Kimberlee Hancill,"Soup - Knorr, Country Bean",81.64,10/20/2022
1005,499,Anet Scaife,Soup - French Onion,98.51,10/20/2022


In [23]:
from pyspark.sql.functions import isnan, when, count, col

# Load your DataFrame
df_qc_sales_new = spark.read.option("branch", "ETL_0405") \
        .format("iceberg") \
        .load("glue.test.salesnew") \

# Check for NULL values in all columns
null_counts_new = df_qc_sales_new.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_qc_sales_new.columns]).collect()[0]

# Check if any columns have NULL values
if any([null_counts_new[i] > 0 for i in range(len(null_counts_new))]):
    print("There are NULL values in the DataFrame.")
else:
    print("There are no NULL values in the DataFrame.")


There are no NULL values in the DataFrame.


# Publish Stage:

In [37]:
spark.sql("SELECT * FROM glue.test.salesnew.refs").toPandas()

,name,type,snapshot_id,max_reference_age_in_ms,min_snapshots_to_keep,max_snapshot_age_in_ms
0,ETL_0305,BRANCH,5073925010883267751,NaN,NaN,NaN
1,main,BRANCH,3393406712675561257,NaN,NaN,NaN


In [38]:
spark.sql("CALL glue.system.cherrypick_snapshot('test.salesnew', 5073925010883267751)").show()

23/05/04 19:28:12 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up
+-------------------+-------------------+
| source_snapshot_id|current_snapshot_id|
+-------------------+-------------------+
|5073925010883267751|5073925010883267751|
+-------------------+-------------------+



In [39]:
spark.sql("SELECT * FROM glue.test.salesnew VERSION AS OF 'main'").toPandas()

,id,name,product,price,date
0,498,Elston Bage,Halibut - Fletches,20.61,10/21/2022
1,499,Daryl Maffioni,Cornstarch,44.13,10/21/2022
2,500,Glad Berre,Sour Puss Sour Apple,45.91,10/21/2022
3,501,Marjy Riha,"Beef - Ox Tail, Frozen",20.42,10/21/2022
4,502,Augie Hancill,Tomatoes - Hot House,19.02,10/21/2022
...,...,...,...,...,...
998,496,Winny McGlone,Pasta - Fusili Tri - Coloured,67.78,10/20/2022
999,497,Hadleigh Ellinor,Oats Large Flake,58.19,10/20/2022
1000,498,Kimberlee Hancill,"Soup - Knorr, Country Bean",81.64,10/20/2022
1001,499,Anet Scaife,Soup - French Onion,98.51,10/20/2022


In [41]:
spark.conf.unset('spark.wap.branch')

In [ ]:
spark.sql("ALTER TABLE glue.test.salesnew DROP BRANCH ETL_0405")